# An overview of BiobOx

Welcome! In this notebook we will overview its main features, and point to relevant publications where they were used. BiobOx is developed by the Degiacomi group (www.degiacomi.org) in Durham University (UK), and is downloadable from Github, https://github.com/degiacom/biobox. Its API is available here: https://degiacom.github.io/biobox/

This overview covers the following topics:

* [Selecting atoms from a (multi)PDB](#select)
* [Protein conformations clustering](#cluster)
* [Protein polyhedral assemblies](#polyhedra)
* [Super-coarse grain modelling](#supercg)
* [Density map cutoff via Collision Cross Section](#density)
* [Calculating cross-linking distances](#xlink)


<div class="alert alert-info">
A publication about BiobOx is currently in preparation. In the meantime, if you use BiobOx in your work, please reference the Github repository.
</div>

Let's get starting by importing BiobOx along with the other packages needed in this notebook.

In [2]:
#import sys
#sys.path.append('.')

import numpy as np
import biobox as bb

## Selecting atoms from a (multi)PDB <a class="anchor" id="select"></a>

Let’s load a protein in a PDB file, containing 20 frames of a molecular dynamics simulation of a small heat-shock protein.

In [14]:
M = bb.Molecule()
M.import_pdb("HSP.pdb")

All the atomic coordinates are stored in `M.coordinates`, which is a directly accessible 3D array of size `(nb_conformations, nb_atoms, 3)`

In [13]:
print(M.coordinates.shape)

(20, 1351, 3)


The properties of all atoms (i.e. anything that is not coordinates) are stored in the pandas data structure `M.data`.

In [14]:
print(M.data)

      atom  index name resname chain  resid  beta  occupancy atomtype  radius  \
0     ATOM      0    N     VAL     X      1   0.0        0.0              1.8   
1     ATOM      1   CA     VAL     X      1   0.0        0.0              1.8   
2     ATOM      2   CB     VAL     X      1   0.0        0.0              1.8   
3     ATOM      3  CG1     VAL     X      1   0.0        0.0              1.8   
4     ATOM      4  CG2     VAL     X      1   0.0        0.0              1.8   
5     ATOM      5    C     VAL     X      1   0.0        0.0              1.8   
6     ATOM      6    O     VAL     X      1   0.0        0.0              1.8   
7     ATOM      7    N     SER     X      2   0.0        0.0              1.8   
8     ATOM      8   CA     SER     X      2   0.0        0.0              1.8   
9     ATOM      9   CB     SER     X      2   0.0        0.0              1.8   
10    ATOM     10   OG     SER     X      2   0.0        0.0              1.8   
11    ATOM     11    C     S

Let's check how many chains this structure is made of:

In [15]:
print(np.unique(M.data["chain"]))

['X']


Well that's odd, small heat shock proteins come as a dimer... As it sometimes happens,the MD engine has replaced the chain assignment with a single symbol, `X`. Let's ask BiobOx to figure out how to split the protein in its two chains.

In [16]:
M.guess_chain_split()
print(np.unique(M.data["chain"]))

['A' 'B']


Ok, that's better, two chains have been found and assigned. Now, let's identify only the backbone atoms of chain A. `atomselect` accepts as parameters single strings, lists or `“*”` as wildcard. After this call, `pos` contains the coordinates of all selected atoms, and `idx` their indices. Another way to select atoms, is to use the `query` method. The following call will yield the same result as the `atomselect` above. The `query` method follows the pandas query syntax, and allows to be more expressive. Any column stored in `M.data` can be addressed. 

In [17]:
pos, idx = M.atomselect("A", "*", ["CA","C","N","O"], get_index=True)
pos, idx = M.query('chain == "A" and name == ["CA","C","N","O"]', get_index=True)

Now that we have identified indices of interest, we can save a subset of the initial pdb in a new one, or to create a new `Molecule` object containing only them.

In [18]:
M.write_pdb("chainA.pdb", index=idx)
M2 = M.get_subset(idx)

multiple conformations may be available in the PDB. By default, the first one is set as current. Is is possible to set as current another one as follows:

In [19]:
M.set_current(2)
pos2, idx2 = M.atomselect("A", "*", ["CA","C","N","O"], get_index=True)

After this new `atomselect` call, `idx2` will be equal to `idx1` (atom selected are still the same), but `pos2` will be different from `pos` (atoms positions differ between different conformations). Unless otherwise specified, `get_subset` selects all the alternative conformations from the atoms of interest. `get_subset` can however also be instructed to select a subset of conformations. For instance, the following call will select only the conformations 0, 1 and 2 of atoms of interest.

In [20]:
M2 = M.get_subset(idx, conformations=[0,1,2])

## protein conformations clustering <a class="anchor" id="cluster"></a>

BiobOx methods return numpy arrays. This means that you can directly benefit from data analysis tools in all major Python scientific computing packages. For instance, let's run a hierarchical clustering on the multi-PDB we previously loaded by first calculating an all-vs-all RMSD matrix.

In [23]:
from scipy.spatial import hierarchy

dist = M.rmsd_distance_matrix(flat=True)
hierarchic_cluster = hierarchy.linkage(dist, method='single')
flat_clusters = hierarchy.fcluster(hierarchic_cluster, 2.0, criterion='distance')

ImportError: cannot import name hierarchy

## protein polyhedral assemblies <a class="anchor" id="polyhedra"></a>

We want to produce several protein tetrahedral assemblies, and compare them to each other. Now, let’s create a `Multimer` arranged according to a tetrahedral symmetry. To do so, we have to load information about the tetrahedral scaffold BiobOx will exploit to align six monomers. By default this information is stored in the file `classes/polyhedron_database.dat` (along with many more symmetries), though the user can import their own database.

In [24]:
P = bb.Multimer()
P.setup_polyhedron('Tetrahedron', M)
P.generate_polyhedron(10, 180, 20, 10)

Now, `P` contains six proteins arranged as a tetrahedron having a radius of 10 Angstrom. Every subunit is rotated with respect of its specific position on the scaffold. Rotation angles are defined with respect of the molecule’s principal axes. Here, we rotate by 180 degrees around the first principal axis, 20 around the second, and 10 around the third. Let’s now build two new polyhedra with different radii and rotation angles:

In [25]:
P.generate_polyhedron(10, 180, 50, 65, add_conformation=True)
P.generate_polyhedron(12, 185, 40, 60, add_conformation=True)

Since we set `add_conformation=True`, the atoms arrangement of the new multimers will be appended as new conformations. With `add_conformation=False` (default) the previous subunits arrangements gets overwritten. Note that assemblies’ multiple conformations are treated by appending on each subunit its different conformation. BiobOx then sets on all subunits the same current position. 

Now, we want to calculate the RMSD between the created multimers’ alpha carbons. With these lines, `dist_mat` will contain the RMSD distance matrix between the multimers:

In [26]:
idxs = P.atomselect("*", "*" ,"*", "CA", get_index=True)[1]
dist_mat = P.rmsd_distance_matrix(points_indices=idxs)

Note that, as for the case of `atomselect` applied to `Molecule` objects, a `query` method is also available. The same selection as the command above can be obtained with:

In [27]:
idx = P.query('name == "CA"', get_index=True)[1]

To select atoms from some specific units, the following command can be issued:

In [28]:
idx = P.query('unit == ["0", "3", "5"] and name == "CA"', get_index=True)

Subunits can also be grouped, and different groups can be rotated differently. In the following example, the tetrahedron’s chains A, B, C and D, E, F form different groups that are rotated independently.

In [29]:
P.conn_type = np.array([0, 0, 0, 1, 1, 1])
P.generate_polyhedron(10, np.array([90,180]), np.array([0,0]), np.array([0,0]))

Note that when more than one edge type is provided, rotation angles should be in the form of a numpy array having the same length as the amount of different groups in connection (values in `conn_type` are used to index the angles arrays).

Polyhedral scaffolds are constituted of vertices connected by edges. By altering the position of the vertices, the scaffolds can be deformed (e.g. useful to model near-symmetries). In BiobOx, deformations are treated in terms of deformation vectors, i.e. unit-vectors indicating in which direction a vertex can move. Here, we will allow the first vertex to move radially. We will then build a tetrahedron, where this vertex is displaced from its initial position by its deformation vector, scaled by a constant (here, 5).

In [30]:
P.add_deformation(0)
P.generate_polyhedron(10, np.array([90,180]), np.array([0,0]), np.array([0,0]), deformation=[5])

Note that add_deformation also accepts user-defined deformation vectors. To see how your scaffold looks like, a pdb file containing the vertices and an associated TCL script for VMD (drawing colored edges, as a function of grouping) can be produced.

In [31]:
P.write_poly_architecture("architecture", scale=10, deformation=[5])

This will generate two files `architecture.pdb` and `architecture.tcl`. The initial unit-sized scaffold will scaled by 10, and the first vertex moved away radially.

<div class="alert alert-info">
    <b>See also</b> : this method was used to build polyhedral assemblies consistent with experimental data in
  <a href="https://www.jbc.org/article/S0021-9258(20)31083-8/fulltext">I. Santhanagopalan I. et al., It takes a dimer to tango: Oligomeric small heat shock proteins dissociate to capture substrate, Journal of Biological Chemisty, 2018 </a>.
</div>

## Super-coarse grain modelling <a class="anchor" id="supercg"></a>

In this example, we will arrange a group of cylinders in a ring. To do so, we have first to create a single collection of points arranged like a cylinder. Unless otherwise specified (using the optional keyword radius), every point composing the `Cylinder` instance (and any other convex point cloud) will have a radius of 1.4 Angstrom. To simulate a smooth surface, one can either increase the points radius, or their density. Here, we will use default values, and the resulting cylinder will then be rotated by 45 degrees along the x axis.

In [32]:
cylinder_length = 20
cylinder_radius = 10
C = bb.Cylinder(cylinder_length, cylinder_radius)
C.rotate(45, 0, 0)

We will now create an assembly loading ten copies of our template cylinder, arrange them in a 30 Angstrom-wide circle, and save the resulting structure into a PDB file.

In [33]:
A = bb.Assembly()
A.load(C, 10)
A.make_circular_symmetry(30)
A.write_pdb("assembly.pdb")

We can now assess some of the assembly’s characteristics, for instance its height and width. This can be done by extracting all the assembly’s points coordinates in a unique numpy array.

In [34]:
xyz = A.get_all_xyz()
width = np.max(xyz[:, 0]) - np.min(xyz[:, 0])
height = np.max(xyz[:, 2]) - np.min(xyz[:, 2])

An alternative way to measure assembly dimensions, it to profit of methods in `Structure` class. Here we collapse the `Assembly` units coordinates in a single `Structure` instance.

In [35]:
S = A.make_structure()
print(S.get_size())

[ 135.6         133.29258924   32.24406922]


In case not all the subunits of the assembly are the same, a list of subunits can be loaded. In this case, we will load a `Sphere` (and call it `S`) as well as two identical cylinders (called `C1` and `C2`).

In [36]:
sphere_radius = 20
cylinder_radius = 5
cylinder_length = 50

S = bb.Sphere(sphere_radius)
C = bb.Cylinder(cylinder_radius, cylinder_length)
A2 = bb.Assembly()
A2.load_list([S, C, C], ["S", "C1", "C2"])

Now, we will arrange the three loaded structures so that the bases of two cylinders are in touch with the sphere, and one cylinder is rotated by 45 degrees with respect to the other.

In [37]:
A2.translate(0, 0, -cylinder_length/2.0-sphere_radius, ["C1", "C2"])
A2.rotate(0.0, 45.0, 180.0, ["C2"])

As you can see, translations (and rotations) can be applied to units subsets. In this case, we kept the sphere fixed, and only translated the cylinders, and then rotated just one of the two cylinders.

<div class="alert alert-info">
    <b>See also</b> :  this super-coarse grain approach was exploited to calculate the collision cross-section of curved chains of ellipsoids in Fig.3 of
  <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4832279/">M. A. McDowell et al., Characterisation of Shigella Spa33 and Thermotoga FliM/N reveals a new model for C-ring assembly in T3SS, Molecular Microbiology, 2015</a>.
</div>

<div class="alert alert-info">
    <b>See also</b> :  a graphical representation of typical membrane protein arrangements was obtained combining super-coarse grain models and VMD-generated lipid bilayers, Fig.3 of  
  <a href="https://pubs.acs.org/doi/abs/10.1021/jacs.5b00420">C. Bechara and C. V. Robinson, Different Modes of Lipid Binding to Membrane Proteins Probed by Mass Spectrometry, JACS, 2015</a>.
</div>

## Density map cutoff via Collision Cross Section <a class="anchor" id="density"></a>

Ion Mobility (IM) experiments report on a molecule’s collision cross section (CCS). Here we show how to relate IM data with a electron density 3D reconstruction obtained by Electron Microscopy (EM). 
We first import a GroEL density map `EMD-1800.mrc`.

In [3]:
D = bb.Density()
D.import_map("EMD-1080.mrc", "mrc")

Depending on which threshold value one selects, the resulting isosurface will have a certain volume and CCS. We now compute the map’s relationship between threshold, volume and CCS with 100 equally spaced threshold values. This might take several minutes, depending on map size (by default, a scan between minimal and maximal map intensity is performed). Obtained values will be returned in a numpy array containining as columns `[threshold, volume, CCS]`. This will also be stored in `self.properties[‘scan’]`, for future usage.

In [4]:
tvc = D.threshold_vol_ccs(low=0, sampling_points=100)

placed!
thresh: 0.0, vol=1920569.37623, ccs=29944129536.0 (97575 points)
placed!
thresh: 0.0736101986131, vol=1821485.12064, ccs=26937063424.0 (92541 points)
placed!
thresh: 0.147220397226, vol=1763400.56794, ccs=25223612416.0 (89590 points)
placed!
thresh: 0.220830595839, vol=1719901.12319, ccs=23948818432.0 (87380 points)
placed!
thresh: 0.294440794452, vol=1683861.53797, ccs=23022968832.0 (85549 points)
placed!
thresh: 0.368050993065, vol=1652467.14233, ccs=22108424192.0 (83954 points)
placed!
thresh: 0.441661191678, vol=1621820.70093, ccs=21338664960.0 (82397 points)
placed!
thresh: 0.515271390291, vol=1593221.29224, ccs=20593584128.0 (80944 points)
placed!
thresh: 0.588881588904, vol=1567928.62866, ccs=19963172864.0 (79659 points)
placed!
thresh: 0.662491787518, vol=1544348.38666, ccs=19349002240.0 (78461 points)
placed!
thresh: 0.736101986131, vol=1521398.00088, ccs=18778869760.0 (77295 points)
placed!
thresh: 0.809712184744, vol=1496439.94842, ccs=18132992000.0 (76027 points)
pl

Let’s predict the density CCS using a fitted mass-based threshold, and compare it the known CCS of 24500 A^2. This requires providing the map’s resolution (here, 5.4 Angstrom) and the mass of GroEL (801 kDa). The procedure interrogates the data previously stored in `D.properties[‘scan’]`.

In [8]:
ccs_mass, fitted_mass_thresh = D.predict_ccs_from_mass(5.4, 801)
print(ccs_mass)
error = 100 * (np.abs(ccs_mass - 24500)/24500.)
print(error)

0.0
100.0


Error should be typically less than 5%. Values greater than 8% indicate that the protein’s conformation is likely different between EM and IM. We can use `fitted_mass_thresh` to create a bead model, that can then be saved into a PDB.

In [ ]:
D.place_points(fitted_mass_thresh)
D.write_pdb("model_ccs_mass.pdb")

<div class="alert alert-info">
    <b>See also</b> :  this method is described in
  <a href="https://pubs.rsc.org/en/Content/ArticleLanding/2016/AN/C5AN01636C#!divAbstract">M. T. Degiacomi and J. L. P. Benesch, EMnIM: software for relating ion mobility mass spectrometry and electron microscopy data, Analyst, 2016 </a>.
</div>

## Calculating cross-linking distances <a class="anchor" id="xlink"></a>

Cross-linking experiments report on the distance between the side chain of specific amino-acids. This distance, measured by a cross-linker molecule, is however not a straight line, but a “shortest solvent accessible path”.

To identify in a structure which lysines may be cross-linked, we start loading it and identifying the location of all lysines’ NZ atoms:

In [49]:
idx = M.atomselect("*", "LYS", "NZ", use_resname=True, get_index=True)[1]
print(M.get_data(idx))

[['ATOM' 229 'NZ' 'LYS' 'A' 28 0.0 0.0 '' 1.8 0.0]
 ['ATOM' 245 'NZ' 'LYS' 'A' 30 0.0 0.0 '' 1.8 0.0]
 ['ATOM' 308 'NZ' 'LYS' 'A' 39 0.0 0.0 '' 1.8 0.0]
 ['ATOM' 468 'NZ' 'LYS' 'A' 57 0.0 0.0 '' 1.8 0.0]
 ['ATOM' 908 'NZ' 'LYS' 'B' 114 0.0 0.0 '' 1.8 0.0]
 ['ATOM' 924 'NZ' 'LYS' 'B' 116 0.0 0.0 '' 1.8 0.0]
 ['ATOM' 987 'NZ' 'LYS' 'B' 125 0.0 0.0 '' 1.8 0.0]
 ['ATOM' 1147 'NZ' 'LYS' 'B' 143 0.0 0.0 '' 1.8 0.0]]


To calculate the path distance between all these atoms, we must first define which protein atoms should be used for clash detection. Here, we select all backbone atoms as well as beta carbon ones. Furthermore, atoms buried in the protein core are also added (with `densify=True`). This makes the protein core more “dense”, reducing the likelihood that a path will find its way through the protein, instead of around it.

In [44]:
XL = bb.Xlink(M)
XL.set_clashing_atoms(atoms=["CA", "C", "N", "O", "CB"], densify=True)

array([ True,  True,  True, ...,  True,  True, False], dtype=bool)

We then set up the grid used by the path detection algorithms. Here, we use a local search, using a cubic moving grid of 18 Angstrom per side. After this, the distance matrix path detection algorithm can be launched. We will use a lazy Theta* method, with flexible side chains, and path smoothing as postprocessing.

In [50]:
XL.setup_local_search(maxdist=18)
distance_mat = XL.distance_matrix(idx, method="theta", smooth=True, flexible_sidechain=True)
print(distance_mat)

[[  0.           1.           3.25631629   1.          -1.          -1.
   10.66411103  -1.        ]
 [  1.           0.           9.32140808   1.          -1.          -1.
   13.74563207  -1.        ]
 [  3.25631629   9.32140808   0.           6.85012831  10.18309944
   13.59413095   2.03474758   6.1051381 ]
 [  1.           1.           6.85012831   0.          -1.          -1.
    7.11064889  -1.        ]
 [ -1.          -1.          10.18309944  -1.           0.           1.
    3.29491726   1.        ]
 [ -1.          -1.          13.59413095  -1.           1.           0.
    9.54715739   1.        ]
 [ 10.66411103  13.74563207   2.03474758   7.11064889   3.29491726
    9.54715739   0.           6.26021856]
 [ -1.          -1.           6.1051381   -1.           1.           1.
    6.26021856   0.        ]]


`distance_mat` is the distance matrix between all lysines, sorted according to `idx`. It will contain -1 for lysine’s linking atoms too far to be encompassed by the moving grid, and -2 for failed path detection (e.g. because a linking atom is buried).

<div class="alert alert-info">
    <b>See also</b> :  this method is presented and benchmarked in
  <a href="https://www.cell.com/structure/fulltext/S0969-2126(17)30291-5"> M. T. Degiacomi et al., Accommodating protein dynamics in the analysis of chemical cross-links, Structure, 2017</a>.
</div>